### Name: Syed Asad Rizvi

### ERP ID: 25365

### Subject: Text Analytics

Importing Libraries

In [233]:
import pandas as pd
import re

import spacy
nlp = spacy.load('en_core_web_md')

import numpy as np
np.set_printoptions(precision=4)
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

import networkx as nx

from numpy import array
from numpy import diag
from numpy import zeros
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

### Wikipedia Articles Dataset

Loading Dataset

In [31]:
df = pd.read_csv('enwiki20170820.csv')
df.head()

,ARTICLE_ID,TITLE,SECTION_TITLE,SECTION_TEXT
0,0,Anarchism,Introduction,\n\n\n\n\n\n'''Anarchism''' is a political phi...
1,0,Anarchism,Etymology and terminology,\n\nThe term ''anarchism'' is a compound word ...
2,0,Anarchism,History,\n\n===Origins===\nWoodcut from a Diggers docu...
3,0,Anarchism,Anarchist schools of thought,\nPortrait of philosopher Pierre-Joseph Proudh...
4,0,Anarchism,Internal issues and debates,\nconsistent with anarchist values is a contro...


Appending the articles from the dataset to a list

In [32]:
articles_list = df['SECTION_TEXT'].to_list()

Taking 15 random articles from the dataset of more than 500 words

In [33]:
def access_elements(articles, list_index):
    result = [articles[i] for i in list_index]
    return result

article_list = []
list_index = [13,52,210,303,379,401,501,565,609,656,705,804,854,900,997]
article_list.append(access_elements(articles_list, list_index))

In [34]:
article_list = [item for sublist in article_list for item in sublist]
article_list

['\n\nIt has long been presumed that there is a common cause at the genetic, cognitive, and neural levels for autism\'s characteristic triad of symptoms. However, there is increasing suspicion that autism is instead a complex disorder whose core aspects have distinct causes that often co-occur.\n\nDeletion (1), duplication (2) and inversion (3) are all chromosome abnormalities that have been implicated in autism.\nAutism has a strong genetic basis, although the genetics of autism are complex and it is unclear whether ASD is explained more by rare mutations with major effects, or by rare multigene interactions of common genetic variants. Complexity arises due to interactions among multiple genes, the environment, and epigenetic factors which do not change DNA sequencing but are heritable and influence gene expression. Many genes have been associated with autism through sequencing the genomes of affected individuals and their parents.\n\nStudies of twins suggest that heritability is 0.7 

In [35]:
len(article_list)

15

## Extractive Summarization

Creating a function to break down the articles into sentences

In [36]:
def articles(list_of_articles):
    global sent_list
    global main_list
    main_list = []
    for i in range(0, len(list_of_articles)):
        sent_list = []
        list_of_articles[i] = re.sub(r'.\n\n', '. ', list_of_articles[i])
        list_of_articles[i] = re.sub(r'.\n', '. ', list_of_articles[i])
        doc = nlp(list_of_articles[i])
        
        # Breaking the articles into sentences
        for sent in doc.sents:                 
            sent_list.append(sent.text)
        main_list.append(sent_list)
        
    print(len(main_list))

In [37]:
articles(article_list)

15


### 01) TF-IDF based summarization

In [38]:
def tfidf_summarizer(article_num, summary_words):
    global tfidf_sum
    num_summary_sentence = summary_words
    tfidfVectorizer = TfidfVectorizer()
    words_tfidf = tfidfVectorizer.fit_transform(article_num)
    print(words_tfidf.shape)

    #sum each row and store it in sent_sum
    sent_sum = words_tfidf.sum(axis=1)

    # Sort the sentences in descending order by the sum of TF-IDF values
    important_sent = np.argsort(sent_sum, axis=0)[::-1]
    
    tfidf_sum_list = []
    for i in range(0, len(article_num)):
        if i in important_sent[:num_summary_sentence]:
            tfidf_sum_list.append(article_num[i])
    tfidf_sum = ' '.join(tfidf_sum_list)
    return tfidf_sum

#### Article 01

In [248]:
%%time

tfidf_summ1 = tfidf_summarizer(main_list[0], 1)
print(tfidf_summ1)

(24, 305)
All known teratogens (agents that cause birth defects) related to the risk of autism appear to act during the first eight weeks from conception, and though this does not exclude the possibility that autism can be initiated or affected later, there is strong evidence that autism arises very early in development.
Wall time: 6 ms


In [42]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[0].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 60 words, while the summary has 51 words.


#### Article 02

In [151]:
%%time

tfidf_summ2 = tfidf_summarizer(main_list[1], 1)
print(tfidf_summ2)

(26, 268)
Troy University was the largest institution in the state in 2010, with an enrollment of 29,689 students across four Alabama campuses (Troy, Dothan, Montgomery, and Phenix City), as well as sixty learning sites in seventeen other states and eleven other countries.
Wall time: 10 ms


In [44]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[1].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 41 words.


#### Article 03

In [152]:
%%time

tfidf_summ3 = tfidf_summarizer(main_list[2], 1)
print(tfidf_summ3)

(48, 386)
Sociocultural anthropology also covers economic and political organization, law and conflict resolution, patterns of consumption and exchange, material culture, technology, infrastructure, gender relations, ethnicity, childrearing and socialization, religion, myth, symbols, values, etiquette, worldview, sports, music, nutrition, recreation, games, food, festivals, and language (which is also the object of study in linguistic anthropology).
Wall time: 11 ms


In [46]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[2].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 52 words.


#### Article 04

In [232]:
%%time

tfidf_summ4 = tfidf_summarizer(main_list[3], 2)
print(tfidf_summ4)

(28, 248)
At the 2009 French Open, Agassi was on hand to present Roger Federer, who completed his Career Grand Slam by winning the tournament and joined Agassi as one of six men to complete the Career Grand Slam, with the trophy. Agassi and Sampras met again one year later for an exhibition match at Madison Square Garden in New York in front of 19 000 spectators as Sampras defeated Agassi in two sets.
Wall time: 540 ms


In [48]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[3].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 57 words, while the summary has 72 words.


#### Article 05

In [234]:
%%time

tfidf_summ5 = tfidf_summarizer(main_list[4], 2)
print(tfidf_summ5)

(39, 369)
Old Major, the old boar on the Manor Farm, summons the animals on the farm together for a meeting, during which he refers to humans as "enemies" and teaches the animals a revolutionary song called "Beasts of England". In a subsequent report, Squealer reports sadly to the animals that Boxer died peacefully at the animal hospital; the pigs hold a festival one day after Boxer's death to further praise the glories of Animal Farm and have the animals work harder by taking on Boxer's ways.
Wall time: 10 ms


In [50]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[4].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 82 words, while the summary has 85 words.


#### Article 06

In [235]:
%%time

tfidf_summ6 = tfidf_summarizer(main_list[5], 2)
print(tfidf_summ6)

(36, 350)
The rough-skinned newt (''Taricha granulosa'') from North America and other members of its genus contain the neurotoxin tetrodotoxin (TTX), the most toxic non-protein substance known and almost identical to that produced by pufferfish. In some species, such as the fire-bellied toad (''Bombina spp.''), the warning colouration is on the belly and these animals adopt a defensive pose when attacked, exhibiting their bright colours to the predator.
Wall time: 7 ms


In [52]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[5].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 68 words, while the summary has 66 words.


#### Article 07

In [236]:
%%time

tfidf_summ7 = tfidf_summarizer(main_list[6], 2)
print(tfidf_summ7)

(55, 306)
In its simplest form, the assumption of unit-treatment additivity states that the observed response  from experimental unit  when receiving treatment  can be written as the sum of the unit's response  and the treatment-effect , that is. If the response variable is expected to follow a parametric family of probability distributions, then the statistician may specify (in the protocol for the experiment or observational study) that the responses be transformed to stabilize the variance.
Wall time: 9 ms


In [54]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[6].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 87 words, while the summary has 73 words.


#### Article 08

In [237]:
%%time

tfidf_summ8 = tfidf_summarizer(main_list[7], 1)
print(tfidf_summ8)

(23, 277)
Several of the sights on her trip inspired her, and they found their way into her poem, including the World's Columbian Exposition in Chicago, the "White City" with its promise of the future contained within its alabaster buildings; the wheat fields of America's heartland Kansas, through which her train was riding on July 16; and the majestic view of the Great Plains from high atop Pikes Peak.
Wall time: 8 ms


In [56]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[7].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 67 words.


#### Article 09

In [238]:
%%time

tfidf_summ9 = tfidf_summarizer(main_list[8], 2)
print(tfidf_summ9)

(34, 300)
In glycine, the simplest amino acid, the R group is a hydrogen atom, but in all other amino acids it is contains one or more carbon atoms bonded to hydrogens, and may contain other elements such as sulfur, oxygen or nitrogen. Aspartic acid, for example, possesses one protonated amine and two deprotonated carboxyl groups, for a net charge of −1 at physiological pH. Fatty acids and fatty acid derivatives are another group of carboxylic acids that play a significant role in biology.
Wall time: 9.01 ms


In [58]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[8].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 70 words, while the summary has 82 words.


#### Article 10

In [239]:
%%time

tfidf_summ10 = tfidf_summarizer(main_list[9], 1)
print(tfidf_summ10)

(21, 289)
The stamp featured a detail of the famous photograph of the Earthrise over the Moon taken by Anders on Christmas Eve, and the words, "In the beginning God ..." Just 18 days after the crew's return to Earth, they were featured during the 1969 Super Bowl pre-game show reciting the Pledge of Allegiance prior to the national anthem being performed by Anita Bryant.
Wall time: 5 ms


In [60]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[9].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 56 words, while the summary has 63 words.


#### Article 11

In [240]:
%%time

tfidf_summ11 = tfidf_summarizer(main_list[10], 2)
print(tfidf_summ11)

(37, 315)
The pronunciation of a language often evolves independently of its writing system, and writing systems have been borrowed for languages they were not designed for, so the degree to which letters of an alphabet correspond to phonemes of a language varies greatly from one language to another and even within a single language. French, with its silent letters and its heavy use of nasal vowels and elision, may seem to lack much correspondence between spelling and pronunciation, but its rules on pronunciation, though complex, are actually consistent and predictable with a fair degree of accuracy.
Wall time: 9.01 ms


In [62]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[10].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 95 words.


#### Article 12

In [242]:
%%time

tfidf_summ12 = tfidf_summarizer(main_list[11], 1)
print(tfidf_summ12)

(29, 303)
The major geographical theorist of the empire was actually a former Swedish prisoner-of-war, taken at the Battle of Poltava in 1709 and assigned to Tobolsk, where he associated with Peter's Siberian official, Vasily Tatishchev, and was allowed freedom to conduct geographical and anthropological studies in preparation for a future book.
Wall time: 7 ms


In [65]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[11].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 50 words.


#### Article 13

In [243]:
%%time

tfidf_summ13 = tfidf_summarizer(main_list[12], 1)
print(tfidf_summ13)

(29, 310)
While nine of these have collectively been nominated a World Heritage Site for their geological value, four of them are considered of "Outstanding Universal Value" based on their cultural and natural criteria: Þingvellir, Iceland; Landscape of the Pico Island Vineyard Culture, Portugal; Gough and Inaccessible Islands, United Kingdom; and Brazilian Atlantic Islands: Fernando de Noronha and Atol das Rocas Reserves, Brazil.
Wall time: 8 ms


In [67]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[12].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 66 words, while the summary has 61 words.


#### Article 14

In [244]:
%%time

tfidf_summ14 = tfidf_summarizer(main_list[13], 2)
print(tfidf_summ14)

(26, 259)
Angola changed from a one-party Marxist-Leninist system ruled by the Popular Movement for the Liberation of Angola (MPLA), in place since independence in 1975, to a multiparty democracy based on a new constitution adopted in 1992.   To a certain degree the new democratic institutions worked, notably the National Assembly, with the active participation of UNITA's and the FNLA's elected MPs  - while José Eduardo dos Santos continued to exercise his functions without democratic legitimation.
Wall time: 5.99 ms


In [245]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[13].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 51 words, while the summary has 74 words.


#### Article 15

In [246]:
%%time

tfidf_summ15 = tfidf_summarizer(main_list[14], 2)
print(tfidf_summ15)

(25, 290)
Between 1895 and 1914 he lived in Switzerland (except for one year in Prague, 1911–12), where he received his academic diploma from the Swiss Federal Polytechnic in Zürich (later the Eidgenössische Technische Hochschule, ETH) in 1900. He was visiting the United States when Adolf Hitler came to power in 1933 and—being Jewish—did not go back to Germany, where he had been a professor at the Berlin Academy of Sciences.
Wall time: 7 ms


In [71]:
words = tfidf_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[14].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 69 words.


--------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx--------------------------------------------

### 02) LSA-based summarization

In [72]:
# Using Sumy library

from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [351]:
def lsa_summarizer(article_num, summary_sentences):
    global lsa_sum
    LANGUAGE = "english"
    parser = PlaintextParser.from_string(article_num, Tokenizer(LANGUAGE))
    lsa_summarizer = LsaSummarizer()
        
    lsa_sum_list = []
    for sentence in lsa_summarizer(parser.document, summary_sentences):
        lsa_sum_list.append(str(sentence))

    lsa_sum = ''.join(lsa_sum_list)
    return lsa_sum

#### Article 01

In [249]:
%%time

lsa_summ1 = lsa_summarizer(main_list[0], 2)
print(lsa_summ1)

["\n\nIt has long been presumed that there is a common cause at the genetic, cognitive, and neural levels for autism's characteristic triad of symptoms.', 'The large number of autistic individuals with unaffected family members may result from spontaneous structural variation\xa0— such as deletions, duplications or inversions in genetic material during meiosis.
None
Wall time: 448 ms


In [75]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[0].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 60 words, while the summary has 51 words.


#### Article 02

In [250]:
%%time

lsa_summ2 = lsa_summarizer(main_list[1], 4)
print(lsa_summ2)

', 'In 2007, over 82\xa0percent of schools made adequate yearly progress (AYP) toward student proficiency under the National No Child Left Behind law, using measures determined by the state of Alabama.", 'The largest educational gains were among people with some college education but without degrees.', 'The rate of school corporal punishment in Alabama is surpassed only by Mississippi and Arkansas.', 'The largest single campus is the University of Alabama, located in Tuscaloosa, with 37,665 enrolled for fall 2016.
None
Wall time: 122 ms


In [77]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[1].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 78 words.


#### Article 03

In [251]:
%%time

lsa_summ3 = lsa_summarizer(main_list[2], 5)
print(lsa_summ3)

', 'There is no hard-and-fast distinction between them, and these categories overlap to a considerable degree.', 'As methodology, ethnography is based upon long-term fieldwork within a community or other research site.', 'Comparison across cultures is a key element of method in sociocultural anthropology, including the industrialized (and de-industrialized) West.', 'Cultures in the Standard Cross-Cultural Sample (SCCS) of world societies are.. \n\n\n Africa 30p.', 'Archaeologists examine these material remains in order to deduce patterns of past human behavior and cultural practices.
None
Wall time: 383 ms


In [81]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[2].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 80 words.


#### Article 04

In [252]:
%%time

lsa_summ4 = lsa_summarizer(main_list[3], 3)
print(lsa_summ4)

', 'On September 5, 2007, he was a surprise guest commentator for the Andy Roddick/Roger Federer US Open quarterfinal.', "At the Camden Wealth Advisors Cup's final in Houston, Agassi beat James Blake for a rematch of their 2005 US Open quarterfinal.', 'This time it was Agassi who came out on top in two straight sets.
None
Wall time: 22 ms


In [83]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[3].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 57 words, while the summary has 55 words.


#### Article 05

In [253]:
%%time

lsa_summ5 = lsa_summarizer(main_list[4], 3)
print(lsa_summ5)

', 'The pigs elevate themselves to positions of leadership and set aside special food items, ostensibly for their personal health.', '(In 1940s England, one way for farms to make money was to sell large animals to a knacker, who would kill the animal and boil its remains into animal glue...', 'Years pass, and the windmill is rebuilt along with construction of another windmill, which makes the farm a good amount of income.', 'The pigs start to resemble humans, as they walk upright, carry whips, and wear clothes.
None
Wall time: 39 ms


In [85]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[4].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 82 words, while the summary has 87 words.


#### Article 06

In [254]:
%%time

lsa_summ6 = lsa_summarizer(main_list[5], 4)
print(lsa_summ6)

', 'The first line of defence in salamanders and frogs is the mucous secretion that they produce.", 'Many amphibians are nocturnal and hide during the day, thereby avoiding diurnal predators that hunt by sight.', 'They have various colourings such as mottled browns, greys and olives to blend into the background.', "Some salamanders adopt defensive poses when faced by a potential predator such as the North American northern short-tailed shrew (''Blarina brevicauda'').
None
Wall time: 45 ms


In [87]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[5].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 68 words, while the summary has 71 words.


#### Article 07

In [255]:
%%time

lsa_summ7 = lsa_summarizer(main_list[6], 5)
print(lsa_summ7)

', 'The assumption of unit treatment additivity  usually cannot be directly falsified, according to Cox and Kempthorne.", 'The randomization-based analysis has the disadvantage that its exposition involves tedious algebra and extensive time.', 'However, when applied to data from non-randomized experiments or observational studies, model-based analysis lacks the warrant of randomization.', "For observational data, the derivation of confidence intervals must use ''subjective'' models, as emphasized by Ronald Fisher and his followers.', 'In practice, "statistical models" and observational data are useful for suggesting hypotheses that should be treated very cautiously by the public.
None
Wall time: 57 ms


In [89]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[6].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 87 words, while the summary has 91 words.


#### Article 08

In [256]:
%%time

lsa_summ8 = lsa_summarizer(main_list[7], 2)
print(lsa_summ8)

', 'The tune came to him while he was on a ferryboat trip from Coney Island back to his home in New York City, after a leisurely summer day in 1882, and he immediately wrote it down.', 'Ward\'s music combined with Bates\'s poem were first published together in 1910 and titled "America the Beautiful".
None
Wall time: 21 ms


In [91]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[7].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 54 words.


#### Article 09

In [257]:
%%time

lsa_summ9 = lsa_summarizer(main_list[8], 3)
print(lsa_summ9)

', "Nucleic acids contain the genetic code that determines many of an organism's characteristics, and is passed from parents to offspring.', 'In glycine, the simplest amino acid, the R group is a hydrogen atom, but in all other amino acids it is contains one or more carbon atoms bonded to hydrogens, and may contain other elements such as sulfur, oxygen or nitrogen.', 'Cell membranes are generally impermeable to charged or large, polar molecules because of the lipophilic fatty acyl chains comprising their interior.
None
Wall time: 33 ms


In [93]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[8].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 70 words, while the summary has 83 words.


#### Article 10

In [258]:
%%time

lsa_summ10 = lsa_summarizer(main_list[9], 3)
print(lsa_summ10)

", 'Apollo 11 astronaut Michael Collins said, "Eight\'s momentous historic significance was foremost"; while many space historians, such as Robert K. Poole, see Apollo 8 as the most historically significant of all the Apollo missions.', "Apollo 8 commemorative stampMadalyn Murray O'Hair, an atheist, later caused controversy by bringing a lawsuit against NASA over the reading from Genesis.", "O'Hair wished the courts to ban American astronauts—who were all government employees—from public prayer in space.
None
Wall time: 22 ms


In [95]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[9].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 56 words, while the summary has 73 words.


#### Article 11

In [259]:
%%time

lsa_summ11 = lsa_summarizer(main_list[10], 4)
print(lsa_summ11)

', 'Languages may fail to achieve a one-to-one correspondence between letters and sounds in any of several ways.', '*', 'A language may spell some words with unpronounced letters that exist for historical or other reasons.", 'In standard Spanish, one can tell the pronunciation of a word from its spelling, but not vice versa, as certain phonemes can be represented in more than one way, but a given letter is consistently pronounced.', 'For English, this is partly because the Great Vowel Shift occurred after the orthography was established, and because English has acquired a large number of loanwords at different times, retaining their original spelling at varying levels.
None
Wall time: 30 ms


In [105]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[10].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 107 words.


#### Article 12

In [260]:
%%time

lsa_summ12 = lsa_summarizer(main_list[11], 2)
print(lsa_summ12)

", 'The Russians were enthusiastic about the concept, which allowed them to keep their European identity in geography.', 'The terms Southeast Asia and Oceania, devised in the 19th century, have had several vastly different geographic meanings since their inception.
None
Wall time: 38 ms


In [107]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[11].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 39 words.


#### Article 13

In [261]:
%%time

lsa_summ13 = lsa_summarizer(main_list[12], 3)
print(lsa_summ13)

', 'The MAR produces basaltic volcanoes in Eyjafjallajökull, Iceland, and pillow lava on the ocean floor.Continental shelves in the Atlantic are wide off Newfoundland, southern-most South America, and north-eastern Europe.', 'The deep ocean floor is thought to be fairly flat with occasional deeps, abyssal plains, trenches, seamounts, basins, plateaus, canyons, and some guyots.
None
Wall time: 32 ms


In [109]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[12].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 66 words, while the summary has 53 words.


#### Article 14

In [262]:
%%time

lsa_summ14 = lsa_summarizer(main_list[13], 4)
print(lsa_summ14)

', 'That same year the first parliamentary and presidential elections were held.", 'A runoff election would have been necessary, but never took place.', 'Daily conditions of life throughout the country and specifically Luanda (population approximately 6 million) mirror the collapse of administrative infrastructure as well as many social institutions.', 'The ongoing grave economic situation largely prevents any government support for social institutions.
None
Wall time: 24 ms


In [111]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[13].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 51 words, while the summary has 62 words.


#### Article 15

In [263]:
%%time

lsa_summ15 = lsa_summarizer(main_list[14], 3)
print(lsa_summ15)

', "Einstein's work is also known for its influence on the philosophy of science.', 'He later taught there at the same institute as a professor of theoretical physics between 1912 and 1914 before he left for Berlin.', 'Einstein\'s intellectual achievements and originality have made the word "Einstein" synonymous with "genius".']
None
Wall time: 21 ms


In [113]:
words = lsa_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[14].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 50 words.


-----------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx--------------------------------------------

### 03) Text Rank-based summarization

In [114]:
# Using Sumy Library

from sumy.summarizers.text_rank import TextRankSummarizer

In [352]:
def textrank_sum(article_num, summary_sentences):
    global tr_sum
    LANGUAGE = "english"
    parser = PlaintextParser.from_string(article_num, Tokenizer(LANGUAGE))
    tr_summarizer = TextRankSummarizer()
    
    tr_sum_list = []
    for sentence in tr_summarizer(parser.document, summary_sentences):
        tr_sum_list.append(str(sentence))

    tr_sum = ''.join(tr_sum_list)
    return tr_sum

#### Article 01

In [264]:
%%time

text_summ1 = textrank_sum(main_list[0], 1)
print(text_summ1)

', 'All known teratogens (agents that cause birth defects) related to the risk of autism appear to act during the first eight weeks from conception, and though this does not exclude the possibility that autism can be initiated or affected later, there is strong evidence that autism arises very early in development.
None
Wall time: 59 ms


In [117]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[0].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 60 words, while the summary has 52 words.


#### Article 02

In [265]:
%%time

text_summ2 = textrank_sum(main_list[1], 1)
print(text_summ2)

", 'In the state are four medical schools (as of fall 2015)', '(University of Alabama School of Medicine, University of South Alabama and Alabama College of Osteopathic Medicine and The Edward Via College of Osteopathic Medicine – Auburn Campus), two veterinary colleges (Auburn University and Tuskegee University), a dental school (University of Alabama School of Dentistry), an optometry college (University of Alabama at Birmingham), two pharmacy schools (Auburn University and Samford University), and five law schools (University of Alabama School of Law, Birmingham School of Law, Cumberland School of Law, Miles Law School, and the Thomas Goode Jones School of Law).
None
Wall time: 31 ms


In [119]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[1].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 101 words.


#### Article 03

In [266]:
%%time

text_summ3 = textrank_sum(main_list[2], 2)
print(text_summ3)

', "Anthropology builds upon knowledge from natural sciences, including the discoveries about the origin and evolution of ''Homo sapiens'', human physical traits, human behavior, the variations among different groups of humans, how the evolutionary past of ''Homo sapiens'' has influenced its social organization and culture, and from social sciences, including the organization of human social and cultural relations, institutions, social conflicts, etc.', "Cultural anthropology is the comparative study of the manifold ways in which people ''make sense'' of the world around them, while social anthropology is the study of the ''relationships'' among persons and groups.
None
Wall time: 126 ms


In [121]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[2].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 95 words.


#### Article 04

In [267]:
%%time

text_summ4 = textrank_sum(main_list[3], 2)
print(text_summ4)

', 'At the 2009 French Open, Agassi was on hand to present Roger Federer, who completed his Career Grand Slam by winning the tournament and joined Agassi as one of six men to complete the Career Grand Slam, with the trophy.', 'The rivalry between the former champions headlined sports media again in March 2010 after the two participated in the "Hit for Haiti" charity event organized to raise money for the victims of the earthquake.
None
Wall time: 33 ms


In [123]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[3].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 57 words, while the summary has 75 words.


#### Article 05

In [268]:
%%time

text_summ5 = textrank_sum(main_list[4], 2)
print(text_summ5)

['Old Major, the old boar on the Manor Farm, summons the animals on the farm together for a meeting, during which he refers to humans as "enemies" and teaches the animals a revolutionary song called "Beasts of England".", "In a subsequent report, Squealer reports sadly to the animals that Boxer died peacefully at the animal hospital; the pigs hold a festival one day after Boxer's death to further praise the glories of Animal Farm and have the animals work harder by taking on Boxer's ways.
None
Wall time: 52 ms


In [125]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[4].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 82 words, while the summary has 85 words.


#### Article 06

In [269]:
%%time

text_summ6 = textrank_sum(main_list[5], 3)
print(text_summ6)

', "The rough-skinned newt (''Taricha granulosa'') from North America and other members of its genus contain the neurotoxin tetrodotoxin (TTX), the most toxic non-protein substance known and almost identical to that produced by pufferfish.', 'Some frogs and toads are toxic, the main poison glands being at the side of the neck and under the warts on the back.''), the warning colouration is on the belly and these animals adopt a defensive pose when attacked, exhibiting their bright colours to the predator.
None
Wall time: 52 ms


In [127]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[5].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 68 words, while the summary has 81 words.


#### Article 07

In [270]:
%%time

text_summ7 = textrank_sum(main_list[6], 3)
print(text_summ7)

', "In its simplest form, the assumption of unit-treatment additivity states that the observed response  from experimental unit  when receiving treatment  can be written as the sum of the unit's response  and the treatment-effect , that is.', 'additivity)', 'and uses the randomization procedure of the experiment..', 'Both these analyses require homoscedasticity, as an assumption for the normal-model analysis and as a consequence of randomization and additivity for the randomization-based analysis.', 'If the response variable is expected to follow a parametric family of probability distributions, then the statistician may specify (in the protocol for the experiment or observational study) that the responses be transformed to stabilize the variance.
None
Wall time: 99 ms


In [129]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[6].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 87 words, while the summary has 107 words.


#### Article 08

In [271]:
%%time

text_summ8 = textrank_sum(main_list[7], 1)
print(text_summ8)

', 'Several of the sights on her trip inspired her, and they found their way into her poem, including the World\'s Columbian Exposition in Chicago, the "White City" with its promise of the future contained within its alabaster buildings; the wheat fields of America\'s heartland Kansas, through which her train was riding on July 16; and the majestic view of the Great Plains from high atop Pikes Peak.
None
Wall time: 23 ms


In [131]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[7].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 68 words.


#### Article 09

In [272]:
%%time

text_summ9 = textrank_sum(main_list[8], 2)
print(text_summ9)

', "An α-amino acid has a central carbon (the α or ''alpha'' carbon) which is covalently bonded to a carboxyl group (thus they are carboxylic acids), an amino group, a hydrogen atom and a variable group.', 'In humans and many other animals, hydrochloric acid is a part of the gastric acid secreted within the stomach to help hydrolyze proteins and polysaccharides, as well as converting the inactive pro-enzyme, pepsinogen into the enzyme, pepsin.
None
Wall time: 61 ms


In [133]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[8].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 70 words, while the summary has 73 words.


#### Article 10

In [273]:
%%time

text_summ10 = textrank_sum(main_list[9], 1)
print(text_summ10)

', 'The stamp featured a detail of the famous photograph of the Earthrise over the Moon taken by Anders on Christmas Eve, and the words, "In the beginning God\xa0..." Just 18 days after the crew\'s return to Earth, they were featured during the 1969 Super Bowl pre-game show reciting the Pledge of Allegiance prior to the national anthem being performed by Anita Bryant.']
None
Wall time: 25 ms


In [135]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[9].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 56 words, while the summary has 63 words.


#### Article 11

In [274]:
%%time

text_summ11 = textrank_sum(main_list[10], 2)
print(text_summ11)

', 'In a perfectly phonemic orthography there would be a consistent one-to-one correspondence between the letters and the phonemes, so that a writer could predict the spelling of a word given its pronunciation, and a speaker would always know the pronunciation of a word given its spelling, and vice versa.', 'The pronunciation of a language often evolves independently of its writing system, and writing systems have been borrowed for languages they were not designed for, so the degree to which letters of an alphabet correspond to phonemes of a language varies greatly from one language to another and even within a single language.
None
Wall time: 55 ms


In [137]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[10].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 85 words, while the summary has 103 words.


#### Article 12

In [275]:
%%time

text_summ12 = textrank_sum(main_list[11], 1)
print(text_summ12)

', 'Almost all of Europe and the better part of Asia sit atop the Eurasian Plate, adjoined on the south by the Arabian and Indian Plate and with the easternmost part of Siberia (east of the Chersky Range) on the North American Plate.']
None
Wall time: 37 ms


In [143]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[11].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 62 words, while the summary has 43 words.


#### Article 13

In [276]:
%%time

text_summ13 = textrank_sum(main_list[12], 2)
print(text_summ13)

', 'The MAR rises  above the surrounding ocean floor and its rift valley is the divergent boundary between the North American and Eurasian plates in the North Atlantic and the South American and African plates in the South Atlantic.', 'The MAR is intersected by two perpendicular ridges: the Azores–Gibraltar Transform Fault, the boundary between the Nubian and Eurasian plates, intersects the MAR at the Azores Triple Junction, on either side of the Azores microplate, near the 40°N.
None
Wall time: 49 ms


In [145]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[12].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 66 words, while the summary has 77 words.


#### Article 14

In [277]:
%%time

text_summ14 = textrank_sum(main_list[13], 1)
print(text_summ14)

["\nSince the adoption of a new constitution in 2010, the '''", "politics of Angola''' takes place in a framework of a presidential republic, whereby the President of Angola is both head of state and head of government, and of a multi-party system.
None
Wall time: 27 ms


In [147]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[13].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 51 words, while the summary has 42 words.


#### Article 15

In [278]:
%%time

text_summ15 = textrank_sum(main_list[14], 2)
print(text_summ15)

', 'He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the evolution of quantum theory.', 'Near the beginning of his career, Einstein thought that Newtonian mechanics was no longer enough to reconcile the laws of classical mechanics with the laws of the electromagnetic field.
None
Wall time: 35 ms


In [149]:
words = tr_sum.split()  # split the string into a list of words
num_words = len(words)     # get the length of the list

print("The 10% of original length gives", int(0.1 * len(article_list[14].split())), "words, while the summary has", num_words, "words.")

The 10% of original length gives 54 words, while the summary has 65 words.


----------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx------------------------------------------------

## Abstractive Summarization

### 01) Google/Pegasus-Large Model

In [169]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

In [279]:
from transformers import pipeline
summarizer = pipeline("summarization", model=model, tokenizer=AutoTokenizer.from_pretrained("google/pegasus-large"), truncation=True)

Taking the minimum length of summary to be 25 words less of the 10% of the original document length.

#### Article 01

In [280]:
%%time

google_summ1 = summarizer(article_list[0], min_length=40, do_sample=False)
print(google_summ1)

[{'summary_text': 'Autism has a strong genetic basis, although the genetics of autism are complex and it is unclear whether ASD is explained more by rare mutations with major effects, or by rare multigene interactions of common genetic variants. Hence, a substantial fraction of autism cases may be traceable to genetic causes that are highly heritable but not inherited: that is, the mutation that causes the autism is not present in the parental genome. All known teratogens (agents that cause birth defects) related to the risk of autism appear to act during the first eight weeks from conception, and though this does not exclude the possibility that autism can be initiated or affected later, there is strong evidence that autism arises very early in development.'}]
Wall time: 2min 21s


#### Article 02

In [283]:
%%time

google_summ2 = summarizer(article_list[1], min_length=42, do_sample=False)
print(google_summ2)

[{'summary_text': 'Public primary and secondary education in Alabama is under the purview of the Alabama State Board of Education as well as local oversight by 67 county school boards and 60 city boards of education. In the state are four medical schools (as of fall 2015) (University of Alabama School of Medicine, University of South Alabama and Alabama College of Osteopathic Medicine and The Edward Via College of Osteopathic Medicine – Auburn Campus), two veterinary colleges (Auburn University and Tuskegee University), a dental school (University of Alabama School of Dentistry), an optometry college (University of Alabama at Birmingham), two pharmacy schools (Auburn University and Samford University), and five law schools (University of Alabama School of Law, Birmingham School of Law, Cumberland School of Law, Miles Law School, and the Thomas Goode Jones School of Law).'}]
Wall time: 1min 57s


#### Article 03

In [284]:
%%time

google_summ3 = summarizer(article_list[2], min_length=60, do_sample=False)
print(google_summ3)

[{'summary_text': "Anthropology builds upon knowledge from natural sciences, including the discoveries about the origin and evolution of ''Homo sapiens'', human physical traits, human behavior, the variations among different groups of humans, how the evolutionary past of ''Homo sapiens'' has influenced its social organization and culture, and from social sciences, including the organization of human social and cultural relations, institutions, social conflicts, etc. Cultural anthropology is the comparative study of the manifold ways in which people ''make sense'' of the world around them, while social anthropology is the study of the ''relationships'' among persons and groups. Cultural anthropology is more related to philosophy, literature and the arts (how one's culture affects experience for self and group, contributing to more complete understanding of the people's knowledge, customs, and institutions), while social anthropology is more related to sociology and history. Inquiry in s

#### Article 04

In [285]:
%%time

google_summ4 = summarizer(article_list[3], min_length=32, do_sample=False)
print(google_summ4)

[{'summary_text': 'At the 2009 French Open, Agassi was on hand to present Roger Federer, who completed his Career Grand Slam by winning the tournament and joined Agassi as one of six men to complete the Career Grand Slam, with the trophy. He played the Cancer Treatment Centers of America Tennis Championships at Surprise, Arizona, where he reached the final before bowing to eventual champion Todd Martin. Agassi beat Courier in the final of the Staples Champions Cup in Boston and later defeated Sampras at the CTCA Championships at his hometown Las Vegas.'}]
Wall time: 1min 15s


#### Article 05

In [286]:
%%time

google_summ5 = summarizer(article_list[4], min_length=57, do_sample=False)
print(google_summ5)

[{'summary_text': 'Old Major, the old boar on the Manor Farm, summons the animals on the farm together for a meeting, during which he refers to humans as "enemies" and teaches the animals a revolutionary song called "Beasts of England". When some animals recall the Battle of the Cowshed, Napoleon (who was nowhere to be found during the battle) frequently smears Snowball as a collaborator of Jones\', while falsely representing himself as the hero of the battle.'}]
Wall time: 1min 42s


#### Article 06

In [289]:
%%time

google_summ6 = summarizer(article_list[5], min_length=43, do_sample=False)
print(google_summ6)

[{'summary_text': "The rough-skinned newt (''Taricha granulosa'') from North America and other members of its genus contain the neurotoxin tetrodotoxin (TTX), the most toxic non-protein substance known and almost identical to that produced by pufferfish. The only predators with some tolerance to the poison are certain populations of common garter snake (''Thamnophis sirtalis''). The only predators with some tolerance to the poison are certain populations of common garter snake (''Thamnophis sirtalis'')."}]
Wall time: 1min 26s


#### Article 07

In [290]:
%%time

google_summ7 = summarizer(article_list[6], min_length=62, do_sample=False)
print(google_summ7)

[{'summary_text': "In its simplest form, the assumption of unit-treatment additivity states that the observed response from experimental unit when receiving treatment can be written as the sum of the unit's response and the treatment-effect , that is. The assumption of unit-treatment additivity implies that, for every treatment , the th treatment has exactly the same effect on every experiment unit. For a randomized experiment, the assumption of unit-treatment additivity ''implies'' that the variance is constant for all treatments."}]
Wall time: 1min 9s


#### Article 08

In [291]:
%%time

google_summ8 = summarizer(article_list[7], min_length=29, do_sample=False)
print(google_summ8)

[{'summary_text': 'Several of the sights on her trip inspired her, and they found their way into her poem, including the World\'s Columbian Exposition in Chicago, the "White City" with its promise of the future contained within its alabaster buildings; the wheat fields of America\'s heartland Kansas, through which her train was riding on July 16; and the majestic view of the Great Plains from high atop Pikes Peak. Ward\'s music combined with Bates\'s poem were first published together in 1910 and titled "America the Beautiful". Ward died in 1903, not knowing the national stature his music would attain since the music was only first applied to the song in 1904.'}]
Wall time: 1min 41s


#### Article 09

In [292]:
%%time

google_summ9 = summarizer(article_list[8], min_length=45, do_sample=False)
print(google_summ9)

[{'summary_text': "An -amino acid has a central carbon (the  or ''alpha'' carbon) which is covalently bonded to a carboxyl group (thus they are carboxylic acids), an amino group, a hydrogen atom and a variable group. In glycine, the simplest amino acid, the R group is a hydrogen atom, but in all other amino acids it is contains one or more carbon atoms bonded to hydrogens, and may contain other elements such as sulfur, oxygen or nitrogen. At physiological pH, typically around 7, free amino acids exist in a charged form, where the acidic carboxyl group (-COOH) loses a proton (-COO) and the basic amine group (-NH2) gains a proton (-NH)."}]
Wall time: 1min 50s


#### Article 10

In [293]:
%%time

google_summ10 = summarizer(article_list[9], min_length=31, do_sample=False)
print(google_summ10)

[{'summary_text': "Even though the year saw political assassinations, political unrest in the streets of Europe and America, and the Prague Spring, ''Time'' magazine chose the crew of Apollo 8 as its Men of the Year for 1968, recognizing them as the people who most influenced events in the preceding year. One of the most famous aspects of the flight was the Earthrise picture that was taken as they came around for their fourth orbit of the Moon."}]
Wall time: 1min 36s


#### Article 11

In [294]:
%%time

google_summ11 = summarizer(article_list[10], min_length=60, do_sample=False)
print(google_summ11)

[{'summary_text': 'In a perfectly phonemic orthography there would be a consistent one-to-one correspondence between the letters and the phonemes, so that a writer could predict the spelling of a word given its pronunciation, and a speaker would always know the pronunciation of a word given its spelling, and vice versa. The pronunciation of a language often evolves independently of its writing system, and writing systems have been borrowed for languages they were not designed for, so the degree to which letters of an alphabet correspond to phonemes of a language varies greatly from one language to another and even within a single language.'}]
Wall time: 1min 44s


#### Article 12

In [295]:
%%time

google_summ12 = summarizer(article_list[11], min_length=37, do_sample=False)
print(google_summ12)

[{'summary_text': 'The Don River became unsatisfactory to northern Europeans when Peter the Great, king of the Tsardom of Russia, defeating rival claims of Sweden and the Ottoman Empire to the eastern lands, and armed resistance by the tribes of Siberia, synthesized a new Russian Empire extending to the Ural Mountains and beyond, founded in 1721. The border between the Black Sea and the Caspian is usually placed along the crest of the Arabian Mountains, although it is sometimes placed further north. The Maluku Islands in Indonesia are often considered to lie on the border of southeast Asia, with New Guinea, to the east of the islands, being wholly part of Oceania.'}]
Wall time: 1min 53s


#### Article 13

In [296]:
%%time

google_summ13 = summarizer(article_list[12], min_length=41, do_sample=False)
print(google_summ13)

[{'summary_text': 'The MAR rises above the surrounding ocean floor and its rift valley is the divergent boundary between the North American and Eurasian plates in the North Atlantic and the South American and African plates in the South Atlantic. The MAR is intersected by two ridges: the Azores–Gibraltar Transform Fault, the boundary between the Nubian and Eurasian plates, intersects the MAR at the Azores Triple Junction, on either side of the Azores microplate, near the 40N.'}]
Wall time: 1min 50s


#### Article 14

In [297]:
%%time

google_summ14 = summarizer(article_list[13], min_length=26, do_sample=False)
print(google_summ14)

[{'summary_text': "Since the adoption of a new constitution in 2010, the '''politics of Angola''' takes place in a framework of a presidential republic, whereby the President of Angola is both head of state and head of government, and of a multi-party system. Angola changed from a one-party Marxist-Leninist system ruled by the Popular Movement for the Liberation of Angola (MPLA), in place since independence in 1975, to a multiparty democracy based on a new constitution adopted in 1992. In the presidential elections, President José Eduardo dos Santos won the first round election with more than 49% of the vote to Jonas Savimbi's 40%."}]
Wall time: 2min 9s


#### Article 15

In [298]:
%%time

google_summ15 = summarizer(article_list[14], min_length=29, do_sample=False)
print(google_summ15)

[{'summary_text': 'Einstein developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the evolution of quantum theory. Between 1895 and 1914 he lived in Switzerland (except for one year in Prague, 1911–12), where he received his academic diploma from the Swiss Federal Polytechnic in Zrich (later the Eidgenssische Technische Hochschule, ETH) in 1900.'}]
Wall time: 1min 39s


------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxx-------------------------------------------------------

### 02) Facebook/Bart-Large-Cnn Model

In [25]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model2 = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [299]:
from transformers import pipeline
summarizer = pipeline("summarization", model=model2, tokenizer=AutoTokenizer.from_pretrained("facebook/bart-large-cnn"), truncation=True)

Taking the minimum length of summary to be 25 words less of the 10% of the original document length.

#### Article 01

In [300]:
%%time

bart_summ1 = summarizer(article_list[0], min_length = 40, do_sample=False)
print(bart_summ1)

[{'summary_text': "It has long been presumed that there is a common cause at the genetic, cognitive, and neural levels for autism's characteristic triad of symptoms. There is increasing suspicion that autism is instead a complex disorder whose core aspects have distinct causes that often co-occur."}]
Wall time: 22 s


#### Article 02

In [301]:
%%time

bart_summ2 = summarizer(article_list[1], min_length = 42, do_sample=False)
print(bart_summ2)

[{'summary_text': "Alabama has 14 four-year public universities, two-year community colleges, and 17 private, undergraduate and graduate universities. The largest single campus is the University of Alabama, located in Tuscaloosa, with 37,665 enrolled for fall 2016. According to the 2011 ''U.S. News & World Report'', Alabama had three universities ranked in the top 100 Public Schools in America."}]
Wall time: 27.5 s


#### Article 03

In [302]:
%%time

bart_summ3 = summarizer(article_list[2], min_length = 60, do_sample=False)
print(bart_summ3)

[{'summary_text': 'Anthropology is a global discipline involving humanities, social sciences and natural sciences. Early anthropology originated in Classical Greece and Persia and tried to understand observable cultural diversity. Sociocultural anthropology has been heavily influenced by structuralist and postmodern theories, as well as a shift toward the analysis of modern societies. Ethnology involves the systematic comparison of different cultures.'}]
Wall time: 27.6 s


#### Article 04

In [303]:
%%time

bart_summ4 = summarizer(article_list[3], min_length = 32, do_sample=False)
print(bart_summ4)

[{'summary_text': 'Since retiring in 2006, Agassi has participated in a series of charity tournaments and continues his work with his own charity. He played an exhibition match at Wimbledon, teaming with his wife, Steffi Graf, to play with Tim Henman and Kim Clijsters in the summer of 2009. In 2009, in Macau Agassi and Sampras met for the first time on court since the 2002 US Open final.'}]
Wall time: 29.3 s


#### Article 05

In [304]:
%%time

bart_summ5 = summarizer(article_list[4], min_length = 57, do_sample=False)
print(bart_summ5)

[{'summary_text': 'Old Major, the old boar on the Manor Farm, summons the animals on the farm together for a meeting. When Major dies, two young pigs, Snowball and Napoleon, assume command and consider it a duty to prepare for the Rebellion. The animals revolt and drive the drunken and irresponsible farmer Mr. Jones from the farm, renaming it "Animal Farm" They adopt the Seven Commandments of Animalism, the most important of which is, "All animals are equal".'}]
Wall time: 35.6 s


#### Article 06

In [305]:
%%time

bart_summ6 = summarizer(article_list[5], min_length = 43, do_sample=False)
print(bart_summ6)

[{'summary_text': 'Amphibians have evolved various defence mechanisms to keep themselves alive. The first line of defence in salamanders and frogs is the mucous secretion that they produce. Caecilians have been little studied in this respect, but the Cayenne caecilian produces toxic mucus that has killed predatory fish. Poisonous species often use bright colouring to warn potential predators.'}]
Wall time: 29.1 s


#### Article 07

In [306]:
%%time

bart_summ7 = summarizer(article_list[6], min_length = 62, do_sample=False)
print(bart_summ7)

[{'summary_text': 'The analysis of variance has been studied from several approaches. The most common of which uses a linear model that relates the response to the treatments and blocks. Note that the model is linear in parameters but may be nonlinear across factor levels. Interpretation is easy when data is balanced across factors but much deeper understanding is needed for unbalanced data.'}]
Wall time: 27.6 s


#### Article 08

In [307]:
%%time

bart_summ8 = summarizer(article_list[7], min_length = 29, do_sample=False)
print(bart_summ8)

[{'summary_text': "In 1893, at the age of 33, English professor Elizabeth Bates took a train trip to Colorado Springs, Colorado, to teach a short summer school session. The words of the poem started to come to her, and she wrote them down upon returning to her hotel room at the original Antlers Hotel. The poem was initially published two years later in ''The Congregationalist'' to commemorate the Fourth of July. Several existing pieces of music were adapted to the poem. A hymn tune composed by Samuel A. Ward was generally considered the best music as early as 1910."}]
Wall time: 38.3 s


#### Article 09

In [308]:
%%time

bart_summ9 = summarizer(article_list[8], min_length = 45, do_sample=False)
print(bart_summ9)

[{'summary_text': "Many biologically important molecules are acids. Nucleic acids contain the genetic code that determines many of an organism's characteristics. DNA contains the chemical blueprint for the synthesis of proteins which are made up of amino acid subunits. Cell membranes contain fatty acid esters such as phospholipids. Acid-base equilibrium plays a critical role in regulating mammalian breathing."}]
Wall time: 27.1 s


#### Article 10

In [309]:
%%time

bart_summ10 = summarizer(article_list[9], min_length = 31, do_sample=False)
print(bart_summ10)

[{'summary_text': 'Apollo\xa08 came at the end of 1968, a year that had seen much upheaval in the United States and most of the world. One of the most famous aspects of the flight was the Earthrise picture that was taken as they came around for their fourth orbit of the Moon. It has been credited with a role in inspiring the first Earth Day in 1970. The mission was the most widely covered by the media since the first American orbital flight, Mercury-Atlas 6 by John Glenn in 1962.'}]
Wall time: 31.8 s


#### Article 11

In [310]:
%%time

bart_summ11 = summarizer(article_list[10], min_length = 60, do_sample=False)
print(bart_summ11)

[{'summary_text': 'In a perfectly phonemic orthography there would be a consistent one-to-one correspondence between the letters and the phonemes. This ideal is not usually achieved in practice; some languages (such as Spanish and Finnish) come close to it, while others deviate from it to a much larger degree.'}]
Wall time: 25.7 s


#### Article 12

In [311]:
%%time

bart_summ12 = summarizer(article_list[11], min_length = 37, do_sample=False)
print(bart_summ12)

[{'summary_text': 'Border between Asia and Africa is the Red Sea, the Gulf of Suez, and the Suez Canal. This makes Egypt a transcontinental country, with the Sinai peninsula in Asia and the remainder of the country in Africa. The border between the Black Sea and the Caspian is usually placed along the crest of the Caucasus Mountains.'}]
Wall time: 25.6 s


#### Article 13

In [312]:
%%time

bart_summ13 = summarizer(article_list[12], min_length = 41, do_sample=False)
print(bart_summ13)

[{'summary_text': 'The bathymetry of the Atlantic is dominated by a submarine mountain range called the Mid-Atlantic Ridge. It runs from 87°N or  south of the North Pole to the subantarctic Bouvet Island at 42°S. The deep ocean floor is thought to be fairly flat with occasional deeps, abyssal plains, trenches, seamounts, basins, plateaus and canyons.'}]
Wall time: 35.1 s


#### Article 14

In [313]:
%%time

bart_summ14 = summarizer(article_list[13], min_length = 26, do_sample=False)
print(bart_summ14)

[{'summary_text': 'Angola changed from a one-party Marxist-Leninist system ruled by the Popular Movement for the Liberation of Angola (MPLA) to a multiparty democracy in 1992. The renewal of civil war immediately after the elections, which were considered as fraudulent by UNITA, and the collapse of the Lusaka Protocol, created a split situation.'}]
Wall time: 27.2 s


#### Article 15

In [314]:
%%time

bart_summ15 = summarizer(article_list[14], min_length = 29, do_sample=False)
print(bart_summ15)

[{'summary_text': 'Einstein was a German-born theoretical physicist. He developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics) His work is also known for its influence on the philosophy of science. He received the 1921 Nobel Prize in Physics.'}]
Wall time: 22.4 s


---------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx----------------------------------------------

### Computing Rouge-2 score for each article for each method

Google Pegasus Large Model for Rouge-2 Score

In [364]:
tokenizer_google = AutoTokenizer.from_pretrained("google/pegasus-large")
summarizer_google = pipeline("summarization", model=model, tokenizer=tokenizer_google, truncation=True)

Facebook Bart Model for Rouge-2 Score

In [365]:
tokenizer_bart = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
summarizer_bart = pipeline("summarization", model=model2, tokenizer=tokenizer_bart, truncation=True)

TFIDF function for Rouge-2 score

In [388]:
def tfdf_summarizer_func(article_content, summary_length):
    global sent_list
    sent_list = [sent.text for sent in nlp(article_content).sents]
    tfidfVectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words='english').fit_transform(sent_list)
    tfidf_sum_list = [sent_list[i] for i in range(0, len(sent_list)) if i in np.argsort(tfidfVectorizer.sum(axis=1), axis=0)[::-1][:summary_length]]
    summary = ' '.join(tfidf_sum_list)
    return summary

In [366]:
from rouge_score import rouge_scorer

evaluation_matrix_list = []

for article in article_list:
    tfidf_summary = tfdf_summarizer_func(article, 3)
    lsa_summary = lsa_summarizer(article, 5)
    textrank_summary = textrank_sum(article, 4)
    google_summary = summarizer_google(article, min_length=70, do_sample=False)[0]['summary_text']
    bart_summary = summarizer_bart(article, min_length=70, do_sample=False)[0]['summary_text']

    summary_list = [tfidf_summary, lsa_summary, textrank_summary, google_summary, bart_summary]

    score_matrix = pd.DataFrame()
    columns = ['tf_idf', 'lsa', 'text_rank', 'google-large', 'facebook-bart']

    for each in range(0,len(columns)):
        scorer = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
        rogue = list(scorer.score(summary_list[each], article)['rouge2'])
        score_matrix[columns[each]] = rogue

    score_matrix.set_index(pd.Index(['precision', 'recall', 'f1_measure']), inplace=True)

    evaluation_matrix_list.append(score_matrix)

In [367]:
len(evaluation_matrix_list)

15

Conversion of list to dataframe

In [435]:
evaluation_matrix_scores = pd.concat(evaluation_matrix_list, axis=0)
evaluation_matrix_scores = evaluation_matrix_scores.reset_index().rename(columns={'index': 'Evaluation Metrics'})
evaluation_matrix_scores.index = evaluation_matrix_scores.index + 1
evaluation_matrix_scores

,Evaluation Metrics,tf_idf,lsa,text_rank,google-large,facebook-bart
1,precision,0.221488,0.204959,0.271074,0.198347,0.100826
2,recall,0.985294,0.976378,0.982036,0.983607,0.983871
3,f1_measure,0.361673,0.338798,0.424870,0.330124,0.182909
4,precision,0.185185,0.171296,0.328704,0.203704,0.083333
5,recall,0.983607,0.965217,0.990698,1.000000,0.931034
6,f1_measure,0.311688,0.290957,0.493627,0.338462,0.152975
7,precision,0.175926,0.089120,0.199074,0.186343,0.063657
8,recall,0.987013,0.962500,0.988506,0.987730,0.948276
9,f1_measure,0.298625,0.163136,0.331407,0.313535,0.119306
10,precision,0.162021,0.191638,0.210801,0.153310,0.104530


Saving the evaluation matrix list

In [370]:
import pickle
  
# Open a file and use dump()

with open('evaluation_matrix.pkl', 'wb') as file:
      pickle.dump(evaluation_matrix_list, file)

### Aggregating the 15 matrices for each metric to produce an average ROUGE-2 score matrix across 15 articles

In [437]:
for each in evaluation_matrix_list:
    each['average'] = each.mean(axis=1).values

In [441]:
avg_evaluation_matrix_scores = pd.concat(evaluation_matrix_list, axis=0)
avg_evaluation_matrix_scores = avg_evaluation_matrix_scores.reset_index().rename(columns={'index': 'Evaluation Metrics'})
avg_evaluation_matrix_scores.index = avg_evaluation_matrix_scores.index + 1
avg_evaluation_matrix_scores

,Evaluation Metrics,tf_idf,lsa,text_rank,google-large,facebook-bart,average
1,precision,0.221488,0.204959,0.271074,0.198347,0.100826,0.199339
2,recall,0.985294,0.976378,0.982036,0.983607,0.983871,0.982237
3,f1_measure,0.361673,0.338798,0.424870,0.330124,0.182909,0.327675
4,precision,0.185185,0.171296,0.328704,0.203704,0.083333,0.194444
5,recall,0.983607,0.965217,0.990698,1.000000,0.931034,0.974111
6,f1_measure,0.311688,0.290957,0.493627,0.338462,0.152975,0.317542
7,precision,0.175926,0.089120,0.199074,0.186343,0.063657,0.142824
8,recall,0.987013,0.962500,0.988506,0.987730,0.948276,0.974805
9,f1_measure,0.298625,0.163136,0.331407,0.313535,0.119306,0.245201
10,precision,0.162021,0.191638,0.210801,0.153310,0.104530,0.164460


---------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx--------------------------------------------------

### Computing cosine similarity score for each article for each method

In [414]:
cosine_sim_list = []

for article in article_list:

    tfidf_summary = tfdf_summarizer_func(article, 3)
    lsa_summary = lsa_summarizer(article, 5)
    textrank_summary = textrank_sum(article, 4)
    google_summary = summarizer_google(article, min_length=70, do_sample=False)[0]['summary_text']
    bart_summary = summarizer_bart(article, min_length=70, do_sample=False)[0]['summary_text']

    # tfidf Summarizer
    sent_list = [sent.text for sent in nlp(article).sents]
    tfidfVectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    tfidf_vector = tfidfVectorizer.fit_transform(sent_list).toarray()[0]
    
    summary_list = [tfidf_summary, lsa_summary, textrank_summary, google_summary, bart_summary]

    summary_tfidf_vector = []
    for each in summary_list:
        summary_sent_list = [sent.text for sent in nlp(each).sents]
        summary_tfidf_vector.append(tfidfVectorizer.transform(summary_sent_list))

    score_matrix = pd.DataFrame()
    columns = ['tf_idf', 'lsa', 'text_rank', 'google-large', 'facebook-bart']

    for each in range(0,len(columns)):
        score_matrix[columns[each]] = [cosine_similarity([list(tfidf_vector)],
                                      [list(summary_tfidf_vector[each].toarray()[0])])[0][0]]

    score_matrix.set_index(pd.Index(['cosine_similarity']), inplace=True)
    cosine_sim_list.append(score_matrix)

In [415]:
len(cosine_sim_list)

15

Conversion of list to dataframe

In [442]:
cosine_scores_df = pd.concat(cosine_sim_list, axis=0)
cosine_scores_df.index = [f"cosine_similarity {i+1:02d}" for i in range(len(cosine_scores_df))]

cosine_scores_df = cosine_scores_df.reset_index().rename(columns={'index': 'Cosine similarity metric'})
cosine_scores_df.index = cosine_scores_df.index + 1

cosine_scores_df

,Cosine similarity metric,tf_idf,lsa,text_rank,google-large,facebook-bart
1,cosine_similarity 01,0.067241,1.000000,0.026113,0.154190,1.000000
2,cosine_similarity 02,0.000000,0.106239,0.217695,0.217695,0.000000
3,cosine_similarity 03,0.298894,0.000000,0.282987,0.298894,1.000000
4,cosine_similarity 04,0.179602,0.046293,0.051774,0.051774,0.976701
5,cosine_similarity 05,1.000000,0.000000,1.000000,1.000000,0.740084
6,cosine_similarity 06,0.000000,0.000000,0.000000,0.000000,0.000000
7,cosine_similarity 07,0.177568,0.113647,1.000000,0.113671,1.000000
8,cosine_similarity 08,0.220641,0.000000,0.220641,0.220641,0.000000
9,cosine_similarity 09,0.154792,0.000000,0.123727,0.151211,0.000000
10,cosine_similarity 10,0.222371,0.000000,0.222371,0.222371,1.000000


Saving the cosine similarity score list

In [436]:
# Open a file and use dump()

with open('cosine_similarity_matrix.pkl', 'wb') as file:
      pickle.dump(cosine_sim_list, file)

### Aggregating the 15 matrices to produce an average cosine similarity score matrix across 15 articles

In [443]:
for each_cos in cosine_sim_list:
    each_cos['average'] = each_cos.mean(axis=1).values

In [447]:
avg_cosine_scores_df = pd.concat(cosine_sim_list, axis=0)
avg_cosine_scores_df = avg_cosine_scores_df.reset_index().rename(columns={'index': 'Cosine similarity metric'})
avg_cosine_scores_df['Cosine similarity metric'] = [f'cosine_similarity {i+1:02}' for i in range(len(avg_cosine_scores_df))]
avg_cosine_scores_df.index = avg_cosine_scores_df.index + 1
avg_cosine_scores_df

,Cosine similarity metric,tf_idf,lsa,text_rank,google-large,facebook-bart,average
1,cosine_similarity 01,0.067241,1.000000,0.026113,0.154190,1.000000,0.449509
2,cosine_similarity 02,0.000000,0.106239,0.217695,0.217695,0.000000,0.108326
3,cosine_similarity 03,0.298894,0.000000,0.282987,0.298894,1.000000,0.376155
4,cosine_similarity 04,0.179602,0.046293,0.051774,0.051774,0.976701,0.261229
5,cosine_similarity 05,1.000000,0.000000,1.000000,1.000000,0.740084,0.748017
6,cosine_similarity 06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,cosine_similarity 07,0.177568,0.113647,1.000000,0.113671,1.000000,0.480977
8,cosine_similarity 08,0.220641,0.000000,0.220641,0.220641,0.000000,0.132385
9,cosine_similarity 09,0.154792,0.000000,0.123727,0.151211,0.000000,0.085946
10,cosine_similarity 10,0.222371,0.000000,0.222371,0.222371,1.000000,0.333423


---------------------------------------------xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx-----------------------------------------------